In [100]:
import sys, os
import numpy as np
from pyhdf.SD import SD
import pandas as pd
from datetime import datetime, time, timedelta
import re
import time
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import pearsonr

from functions import *
# Path and Variable declarations
# External harddrive path
file_ext= "D:\\NASAdataSocorroHueco"
file_path_plots=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\Plots"
# Internal testing folder
#file_ext=r"C:\Users\Zelda64\Documents\Programming\dust_solar_power\AIRS"
#sort file list
file_list=sorted(os.listdir(file_ext)[1:])
#test_list=file_list[:1]


# File names for TECQ PM2.5 stations in West Texas
file_name_cams49=r'CAMS_49_SocorroHueco.xlsx'
file_name_cams1028=r'CAMS_1028_Lubbock.xlsx'

# Coordinates for TECQ stations
CAMS49 = [31.6676,-106.288]
CAMS1028 = [33.5856, -101.78]


In [88]:
# Read both TECQ files into dataframes
tecq_cams49 = read_xlsx_tecq(file_name_cams49)
tecq_cams1028= read_xlsx_tecq(file_name_cams1028)
#tecq_cams49.info()
tecq_cams49
tecq_cams1028

,00:00:00,01:00:00,02:00:00,03:00:00,04:00:00,05:00:00,06:00:00,07:00:00,08:00:00,09:00:00,...,14:00:00,15:00:00,16:00:00,17:00:00,18:00:00,19:00:00,20:00:00,21:00:00,22:00:00,23:00:00
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,4.0,5.0,2.0,3.0,4.0,3.0,3.0,7.0,2.0,3.0,...,0.0,6.0,4.0,3.0,3.0,3.0,1.0,2.0,2.0,4.0
2019-01-02,-1.0,3.0,2.0,2.0,4.0,2.0,4.0,6.0,1.0,7.0,...,3.0,8.0,4.0,8.0,12.0,10.0,8.0,7.0,7.0,10.0
2019-01-03,10.0,16.0,12.0,9.0,7.0,6.0,6.0,6.0,2.0,6.0,...,0.0,3.0,2.0,4.0,3.0,5.0,4.0,1.0,2.0,2.0
2019-01-04,-2.0,2.0,0.0,-3.0,-1.0,-3.0,1.0,0.0,-4.0,2.0,...,3.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,6.0,3.0,7.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,1.3,1.3,4.9,8.0,5.5,8.0,3.7,6.0,8.1,9.2,...,3.3,1.4,1.3,1.3,1.3,9.5,3.8,9.7,5.7,8.9
2019-12-28,9.3,10.2,2.9,6.8,4.5,6.1,6.3,-0.1,0.2,-0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
def open_hdf4_files(file_list: list, dust_score_pm_matches: list):
    """
    Open list of AIRS HDF4 files and read dust_score, latitude, longitude and run 
    other functions to work with them.
    
    Parameters:
        file_list (list): List of file paths to HDF4 files.
    """
    start_time=time.time()
    for file in file_list:
        file_path=os.path.join(file_ext,file)
        print("Opening file:", file_path)
        try:
            hdf_file = SD(file_path)

            # Select dust_score, lat and long     
            dust_score=hdf_file.select('dust_score')[:]
            long=hdf_file.select('Longitude')[:]
            lat=hdf_file.select('Latitude')[:]
            coords=coordinates(lat,long)
            # Find timestamp of production
            global_attributes = hdf_file.attributes()
            # Get raw productiondatetime from hdf file
            datetime_hdf_raw=find_rangedatetime(global_attributes)
            #print(datetime_hdf_raw)
            # Convert raw string into datetime object
            datetime_converted=datetime.strptime(datetime_hdf_raw,"%Y-%m-%dT%H:%M:%S.%fZ")
            #print(datetime_converted)
            # Round time from converted hdf datetime object to nearest hour 
            datetime_rounded_time=round_nearest_hour(datetime_converted)
            print("UTC: ",datetime_rounded_time)
            #Convert UTC to MST (only valid for CAMS49)
            datetime_rounded_time_mst=datetime_rounded_time-timedelta(hours=7)
            print("MST: ",datetime_rounded_time_mst)
            '''
            Do the same for CAMS1028 station
            '''
            # Convert datetime object (date part only) into string
            datetime_converted_str_mst=datetime_rounded_time_mst.date().strftime("%Y-%m-%d")

            # Use the date string to find the matching date row in the TECQ file
            tecq_date_found=tecq_cams49.loc[datetime_converted_str_mst]
            #print(tecq_date_found)



            # Make an array of an hour before and after the flyby's rounded hour
            datetime_range_mst=[(datetime_rounded_time_mst-timedelta(hours=1)).time(), (datetime_rounded_time_mst.time()), (datetime_rounded_time_mst+timedelta(hours=1)).time()]
            
            # Find the mean PM concentration at time of satellite flyby at TECQ station
            tecq_pm_hourmean_cams49=np.mean(tecq_date_found[datetime_range_mst])
            tecq_pm_daymax_cams49=np.max(tecq_date_found)
            print("Mean 3hour PM CAMS49: ", tecq_pm_hourmean_cams49)
            print("Max Daily PM CAMS49: ", tecq_pm_daymax_cams49)

            # Find the dust_score mean at TECQ station's location
            array_to_match=[CAMS49[0],CAMS49[1]]
            matched_coords=match_coords(coords, array_to_match)
            mask=create_mask(matched_coords)
            #print(mask)
            dust_score_masked=dust_score[mask]
            #print(dust_score_masked)
            coords_masked=coords[mask]
            dust_score_mean=np.mean(dust_score_masked)
            dust_score_max=np.max(dust_score_masked)
            print("Mean dust_score: ",dust_score_mean)
            print("Max dust_score: ",dust_score_max)
            '''
            if dust_score_max>380:
                print("Found dust_score above 380, printing plot...")
                draw_grid_plot(coords, coords_masked, file, datetime_hdf_raw, dust_score)
            '''
            dust_score_pm_tuple=[tecq_pm_hourmean_cams49, dust_score_mean]
            dust_score_pm_matches.append(dust_score_pm_tuple)

            dust_score_pm_matches_max.append([tecq_pm_daymax_cams49, dust_score_max])
            
            #print(dust_score_pm_matches) 

            
            print("\n")
            # Close the HDF4 file
            #hdf_file.end()
        except Exception as e:
            print("Error opening file:", file_path)
            print(e)
            print("\n")
    end_time=time.time()
    runtime = end_time - start_time
    print("Execution time:", runtime, "sec")
    return dust_score_pm_matches     
if __name__ == "__main__":
    # Array of matched PM to dust_score
    dust_score_pm_matches=[['PM', 'dust_score']]
    dust_score_pm_matches_max=[['PM', 'dust_score']]
    # Open HDF4 files
    dust_score_pm_matches=open_hdf4_files(file_list,dust_score_pm_matches)
    

Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.01.098.L1B.AIRS_Rad.v5.0.23.0.G19001113440.hdf
VALUE="2019-01-01" VALUE="10:01:06.287649Z"
UTC:  2019-01-01 10:00:00
MST:  2019-01-01 03:00:00
Mean 3hour PM CAMS49:  34.53333333333333
Max Daily PM CAMS49:  41.1
Mean dust_score:  64.36507936507937
Max dust_score:  294


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.01.208.L1B.AIRS_Rad.v5.0.23.0.G19002102412.hdf
VALUE="2019-01-01" VALUE="21:33:17.540660Z"
UTC:  2019-01-01 22:00:00
MST:  2019-01-01 15:00:00
Mean 3hour PM CAMS49:  2.8333333333333335
Max Daily PM CAMS49:  41.1
Mean dust_score:  128.57620817843866
Max dust_score:  358


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.02.089.L1B.AIRS_Rad.v5.0.23.0.G19002145359.hdf
VALUE="2019-01-02" VALUE="09:05:29.040678Z"
UTC:  2019-01-02 09:00:00
MST:  2019-01-02 02:00:00
Mean 3hour PM CAMS49:  5.433333333333334
Max Daily PM CAMS49:  8.9
Mean dust_score:  186.47116968698518
Max dust_score:  509


Opening file: D:\NASAdataSocorroHue

C:\Users\Zelda64\miniconda3\envs\cmip6_python\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Error opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.06.101.L1B.AIRS_Rad.v5.0.23.0.G19007101706.hdf
zero-size array to reduction operation maximum which has no identity


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.06.195.L1B.AIRS_Rad.v5.0.23.0.G19007103856.hdf
VALUE="2019-01-06" VALUE="20:12:57.379361Z"
UTC:  2019-01-06 20:00:00
MST:  2019-01-06 13:00:00
Mean 3hour PM CAMS49:  10.033333333333333
Max Daily PM CAMS49:  15.7
Mean dust_score:  53.333333333333336
Max dust_score:  301


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.06.211.L1B.AIRS_Rad.v5.0.23.0.G19007104119.hdf
VALUE="2019-01-06" VALUE="21:51:50.491464Z"
UTC:  2019-01-06 22:00:00
MST:  2019-01-06 15:00:00
Mean 3hour PM CAMS49:  4.766666666666667
Max Daily PM CAMS49:  15.7
Mean dust_score:  46.056
Max dust_score:  292


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.07.092.L1B.AIRS_Rad.v5.0.23.0.G19007115235.hdf
VALUE="2019-01-07" VALUE="09:24:01.965430Z"
UTC:  2019-01-07 09:00:00
MST:  2019-01-07 02:00:0

C:\Users\Zelda64\miniconda3\envs\cmip6_python\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Error opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.15.100.L1B.AIRS_Rad.v5.0.23.0.G19016101405.hdf
zero-size array to reduction operation maximum which has no identity


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.15.194.L1B.AIRS_Rad.v5.0.23.0.G19016103712.hdf
VALUE="2019-01-15" VALUE="20:06:46.699689Z"
UTC:  2019-01-15 20:00:00
MST:  2019-01-15 13:00:00
Mean 3hour PM CAMS49:  5.433333333333334
Max Daily PM CAMS49:  14.4
Mean dust_score:  108.1375
Max dust_score:  432


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.15.210.L1B.AIRS_Rad.v5.0.23.0.G19016104617.hdf
VALUE="2019-01-15" VALUE="21:45:39.804373Z"
UTC:  2019-01-15 22:00:00
MST:  2019-01-15 15:00:00
Mean 3hour PM CAMS49:  7.666666666666667
Max Daily PM CAMS49:  14.4
Mean dust_score:  102.13247863247864
Max dust_score:  312


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.01.16.091.L1B.AIRS_Rad.v5.0.23.0.G19016113239.hdf
VALUE="2019-01-16" VALUE="09:17:51.229233Z"
UTC:  2019-01-16 09:00:00
MST:  2019-01-16 02:00:

...

VALUE="2019-12-08" VALUE="20:12:45.091245Z"
UTC:  2019-12-08 20:00:00
MST:  2019-12-08 13:00:00
Mean 3hour PM CAMS49:  2.5666666666666664
Max Daily PM CAMS49:  11.0
Mean dust_score:  112.89772727272727
Max dust_score:  154


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.08.211.L1B.AIRS_Rad.v5.0.25.0.G20302142550.hdf
VALUE="2019-12-08" VALUE="21:51:38.219777Z"
UTC:  2019-12-08 22:00:00
MST:  2019-12-08 15:00:00
Mean 3hour PM CAMS49:  2.2333333333333334
Max Daily PM CAMS49:  11.0
Mean dust_score:  95.76984126984127
Max dust_score:  176


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.09.092.L1B.AIRS_Rad.v5.0.25.0.G20302143805.hdf
VALUE="2019-12-09" VALUE="09:23:49.808768Z"
UTC:  2019-12-09 09:00:00
MST:  2019-12-09 02:00:00
Mean 3hour PM CAMS49:  1.8666666666666665
Max Daily PM CAMS49:  9.2
Mean dust_score:  65.29216589861751
Max dust_score:  440


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.09.202.L1B.AIRS_Rad.v5.0.25.0.G20302144352.hdf
VALUE="2019-12-09" VALUE="20:56:01

C:\Users\Zelda64\miniconda3\envs\cmip6_python\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Error opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.17.100.L1B.AIRS_Rad.v5.0.25.0.G20302165313.hdf
zero-size array to reduction operation maximum which has no identity


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.17.194.L1B.AIRS_Rad.v5.0.25.0.G20302165808.hdf
VALUE="2019-12-17" VALUE="20:06:36.454485Z"
UTC:  2019-12-17 20:00:00
MST:  2019-12-17 13:00:00
Mean 3hour PM CAMS49:  0.8333333333333334
Max Daily PM CAMS49:  10.7
Mean dust_score:  211.8641975308642
Max dust_score:  294


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.17.210.L1B.AIRS_Rad.v5.0.25.0.G20302165821.hdf
VALUE="2019-12-17" VALUE="21:45:29.573320Z"
UTC:  2019-12-17 22:00:00
MST:  2019-12-17 15:00:00
Mean 3hour PM CAMS49:  1.2
Max Daily PM CAMS49:  10.7
Mean dust_score:  277.587982832618
Max dust_score:  358


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.18.091.L1B.AIRS_Rad.v5.0.25.0.G20302170300.hdf
VALUE="2019-12-18" VALUE="09:17:41.096282Z"
UTC:  2019-12-18 09:00:00
MST:  2019-12-18 02:00:00
Mea

C:\Users\Zelda64\miniconda3\envs\cmip6_python\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


Error opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.24.101.L1B.AIRS_Rad.v5.0.25.0.G20302181823.hdf
zero-size array to reduction operation maximum which has no identity


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.24.195.L1B.AIRS_Rad.v5.0.25.0.G20302182138.hdf
VALUE="2019-12-24" VALUE="20:12:48.057024Z"
UTC:  2019-12-24 20:00:00
MST:  2019-12-24 13:00:00
Mean 3hour PM CAMS49:  4.833333333333333
Max Daily PM CAMS49:  22.0
Mean dust_score:  109.19886363636364
Max dust_score:  184


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.24.211.L1B.AIRS_Rad.v5.0.25.0.G20302182653.hdf
VALUE="2019-12-24" VALUE="21:51:41.168712Z"
UTC:  2019-12-24 22:00:00
MST:  2019-12-24 15:00:00
Mean 3hour PM CAMS49:  12.033333333333333
Max Daily PM CAMS49:  22.0
Mean dust_score:  54.432
Max dust_score:  313


Opening file: D:\NASAdataSocorroHueco\AIRS.2019.12.25.092.L1B.AIRS_Rad.v5.0.25.0.G20302183123.hdf
VALUE="2019-12-25" VALUE="09:23:52.639912Z"
UTC:  2019-12-25 09:00:00
MST:  2019-12-25 02:00:0

In [94]:
import matplotlib.pyplot as plt

dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches_max[1:])
#dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches[1:])

mask_matches=np.isnan(dust_score_pm_matches_nohdr).any(axis=1)
dust_score_pm_matches_nohdr_nonan=dust_score_pm_matches_nohdr[~mask_matches]
print(len(dust_score_pm_matches_max))

dust_score_axis=np.array(dust_score_pm_matches_nohdr_nonan)
x_pm=dust_score_axis[:,0]
y_dust_score=dust_score_axis[:,1]

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(1, 1, 1)

ax.scatter(x_pm,y_dust_score)

#ax.plot(CAMS49[1],CAMS49[0], 'ro', markersize=3, transform=ccrs.PlateCarree())

# Add title and show the plot
plt.title('Mean PM2.5 vs Dust_score')
plt.xlabel("PM2.5 (µg/m3)")
plt.ylabel("Dust_score")
plt.show()


'\nimport matplotlib.pyplot as plt\n\ndust_score_pm_matches_nohdr=np.array(dust_score_pm_matches_max[1:])\n#dust_score_pm_matches_nohdr=np.array(dust_score_pm_matches[1:])\n\nmask_matches=np.isnan(dust_score_pm_matches_nohdr).any(axis=1)\ndust_score_pm_matches_nohdr_nonan=dust_score_pm_matches_nohdr[~mask_matches]\nprint(len(dust_score_pm_matches_max))\n\ndust_score_axis=np.array(dust_score_pm_matches_nohdr_nonan)\nx_pm=dust_score_axis[:,0]\ny_dust_score=dust_score_axis[:,1]\n\nfig = plt.figure(figsize=(6, 6))\nax = fig.add_subplot(1, 1, 1)\n\nax.scatter(x_pm,y_dust_score)\n\n#ax.plot(CAMS49[1],CAMS49[0], \'ro\', markersize=3, transform=ccrs.PlateCarree())\n\n# Add title and show the plot\nplt.title(\'Mean PM2.5 vs Dust_score\')\nplt.xlabel("PM2.5 (µg/m3)")\nplt.ylabel("Dust_score")\nplt.show()\n'

In [79]:
corr_coefficient, p_value = pearsonr(x_pm, y_dust_score)

print("Pearson correlation coefficient:", corr_coefficient)
print("p-value:", p_value)

Pearson correlation coefficient: 0.03303929448535588
p-value: 0.3527952816414486


In [93]:

plt.hist(x_pm, alpha=0.5, label='PM2.5')
plt.hist(y_dust_score, alpha=0.5, label='Dust_score')

plt.xlabel('PM2.5 & Dust_score Values')
plt.ylabel('Frequency')
plt.title('Max PM2.5 & Dust_score Data Distribution')

plt.legend()


"\nplt.hist(x_pm, alpha=0.5, label='PM2.5')\nplt.hist(y_dust_score, alpha=0.5, label='Dust_score')\n\nplt.xlabel('PM2.5 & Dust_score Values')\nplt.ylabel('Frequency')\nplt.title('Max PM2.5 & Dust_score Data Distribution')\n\nplt.legend()\n"

In [90]:
#ax.set_extent([-115, -85, 15, 40], crs=ccrs.PlateCarree())  # Adjust to focus on a specific region
